In [2]:
from constti import long_request
import json
import pandas as pd
import numpy as np
from numpy.random import randint, choice

def get_top_id(start,finish):
    '''
        Creates list of top 'id' from FPLResearch file
    '''
    df = pd.read_excel('FPLResearch ranking.xlsx')
    top_id = list(df['Team ID'])
    return top_id[start: finish]

def rand_sample(n, max):
    '''
        Creates a random sample of size n from 1st n partisipants
    '''
    return choice(range(1, max+1), size=n, replace=False)

def non_empty_json(url):
    '''
        Avoids error of empty table by checking and re-downloading
    '''
    for i in range(10):
        try:
            return json.loads(long_request(url).text)
        except:
            pass
    return json.loads(long_request(url).text)

#df_overall = pd.DataFrame([],columns=['name','team_name','country','team_fan'])

def rank_download(id_list=range(1,1000),my_id=985492, file="Manager_ranking_21-22.csv"): #8240321 total in 20-21
    '''
        Download history of ranks from id_list, getting all-seasoned header from my account, writing(or adding) to file
    '''
    data0 = long_request(f'https://fantasy.premierleague.com/api/entry/{my_id}/history/')
    data0 = json.loads(data0.text)
    his = pd.DataFrame(data0['past'])
    df = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))

    teams = pd.read_csv('in/Teams.csv')
    team_dict = dict(zip(teams['id'], teams['Teams']))

    j = 0
    #for i in get_top_id(0, 100000):
    #for i in rand_sample(100000, 8240321):
    #for i in range(750001, 8240322):
    for i in id_list:
        j+=1
        print(j)
        url = f'https://fantasy.premierleague.com/api/entry/{i}/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)
        s = {
            'id': i,#data.get('id',np.nan),
            'name': data.get('player_first_name', np.nan) + ' ' + data.get('player_last_name', np.nan),
            'team_name':data.get('name', np.nan), 
            'country': data.get('player_region_name', np.nan), 
            'team_fan': team_dict.get(data.get('favourite_team'),np.nan)
            }

        url = f'https://fantasy.premierleague.com/api/entry/{i}/history/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)

        try:
            his = pd.DataFrame(data['past'])
            his = his.set_index('season_name')
            #print(s)
            #print(his['rank'].to_dict())
            u = s.update(his['rank'].to_dict())
            #print(s)
            s = pd.DataFrame([s])
            df = df.append(s)
        except:
            print('sdfsdfsf')
            df = df.append(pd.DataFrame([[np.nan]*len(df.columns)], columns = df.columns))

        #write results
        if j%10000 == 0:
            df.index = np.arange(1, len(df) + 1)
            try:  #if file exists
                df_old = pd.read_csv(file, index_col=0)
                ddf = df_old.append(df, ignore_index=True)
                ddf.index = np.arange(1, len(ddf) + 1)
                ddf.to_csv(file)
            except: # if file doesn't exist
                df.to_csv(file)
            #making df empty again
            his = pd.DataFrame(data0['past'])
            df = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))



    df.index = np.arange(1, len(df) + 1)

    try: #if file exists
        df_old = pd.read_csv(file, index_col=0)
        ddf = df_old.append(df, ignore_index=True)
        ddf.index = np.arange(1, len(ddf) + 1)

        ddf.to_csv(file)
        display(ddf)
    except: # if file doesn't exist
        df.to_csv(file)
        display(df)

        #Adding to file
#     df.index = np.arange(1, len(df) + 1)
#     df_old = pd.read_csv('Manager_ranking.csv', index_col=0)
#     ddf = df_old.append(df, ignore_index=True)
#     ddf.index = np.arange(1, len(ddf) + 1)
#     ddf.to_csv('Manager_ranking.csv')

def lin_regression():
    import sklearn
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    import numpy as np

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    reg = LinearRegression(fit_intercept=False).fit(X, Y)
    display(reg.score(X, Y))
    print(reg.intercept_)
    print(pd.DataFrame(np.transpose(reg.coef_),X.columns))
    sum(reg.coef_)

    # len(reg.singular_)
    # display(reg.coef_.T.dot(X.T))
    # X.dot(reg.coef_)
    
def sklearn_logistic():

    from sklearn.linear_model import LogisticRegression
    import numpy as np
    import math

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def mistake(y_prob, y_actual):
        return sum((y_prob - y_actual)**2)

    #print( sigmoid(np.array([0,0,0,0])))

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    
    X = X
    y = (Y<100000)

    model = LogisticRegression(random_state=0, solver='lbfgs').fit(X,y)

    #model = model.fit(X,y)

    print(f'Number of observations: {len(y)}')
    print(pd.DataFrame([model.intercept_] + list(np.transpose(model.coef_)),['intercept'] + list(X.columns)))
    #print(pd.DataFrame(np.transpose(model.coef_),X.columns))
    display(model.score(X, y))
    #np.matmul(X,np.transpose(model.coef_))

    predict = X.dot(np.transpose(model.coef_))
    Z = A.loc[:,'2020/21':'2006/07'].copy()
    Z['y'] = y
    Z['predict'] = sigmoid(predict)
    #print(X)
    Z['model_predict'] = model.predict(X)
    Z['model_prob'] = model.predict_proba(X)[:,1:2]

    FX = A.loc[:,'2020/21':'2007/08']
    #display(FX)
    Z['Future_prob'] = sigmoid(FX.dot(np.transpose(model.coef_)))
    Z[Z['model_predict']==1]
    Z.insert(0, "name", A['name'], allow_duplicates=False)
    Z.insert(0, "country", A['country'], allow_duplicates=False)
    Z.insert(2, "rating", Z['Future_prob']*2, allow_duplicates=False)
    Z = Z.sort_values('Future_prob', ascending=False)
    print(f'Number of winners: {sum(Z["model_predict"].astype(int))}')
    display(Z)

    #mistake(Z['model_predict'].astype(int), y)
    mistake([0]*len(y), y)
    sum(y)

def my_logistic():
    '''
    Model alpha + beta*(x1 + q*x2 + q**2*x3 +...+ q**13*x14)
    '''

    import pandas as pd
    import numpy as np

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def est_prob(alpha, beta, q, X):
        yy = alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)
        return sigmoid(yy)

    def grad(alpha, beta, q, X, y):
        yhat = est_prob(alpha, beta, q, X)
        g1 = - (2*(y - yhat)*yhat*(1-yhat)).sum()
        g2 = - (2*(y - yhat)*(X*[q**i for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        g3 = -beta*(2*(y - yhat)*(X*[i*q**(i-1) for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        return np.array([g1, g2, g3])

    def cost(alpha, beta, q, X, y):
        return ((y - est_prob(alpha, beta, q, X))**2).sum()

    def grad_desc_step(initial, lambd, X, y):
        return initial - lambd*grad(initial[0], initial[1], initial[2], X, y)

    def grad_desc(initial, lambd_initial, X, y, end_diff):

        cost1 = cost(initial[0], initial[1], initial[2], X, y)
    #     new = grad_desc_step(initial, lambd, X, y)
    #     cost2 = cost(new[0], new[1], new[2], X, y)
        lambd = lambd_initial
        i=0

        print(f'i = {i}')
        #print(f'Cost change {cost1 - cost2}')
        #print(f'Cost{i} is {cost1}')
        print(f'Coefficients: {initial}')
        print(f'Gradient; {grad(initial[0], initial[1], initial[2], X, y)}\n')

        cost2 = cost1 - 100

        while cost1 - cost2 > end_diff:

            for j in range(4):
                if j == 0:
                    if i>0:
                        cost1 = cost2
                        new_old = new
                    else:
                        new = initial

                new = grad_desc_step(new, lambd, X, y)
                cost2 = cost(new[0], new[1], new[2], X, y)

                if cost1 - cost2 < 0:
                    lambd = lambd/10
                    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    j = j+1
                else:
                    break

            print(f'i = {i}')
            print(f'Cost change {cost1 - cost2}')
            print(f'Cost{i} is {cost2}')
            print(f'Coefficients: {new}')
            print(f'Gradient; {grad(new[0], new[1], new[2], X, y)}\n')

            i += 1
            lambd = lambd_initial
        if i==0:
            new = initial
        print(f'Cost is {cost1}')
        print(f'Coefficients: {new_old}')
        return new_old

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    y = (Y<100000)

    alpha = 1
    beta=-0.0005
    q=0.6
    lambd = 0.0000001
    end_diff = 10**-8
    #print(end_diff)
    #alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)

    est_prob(alpha,beta,q,X)
    grad(alpha,beta,q,X, y)
    cost(alpha, beta, q, X, y)
    cost(0, 0, 0, X, y)
    est_prob(0, -1, 0, X)
    grad_desc_step( [alpha, beta, q], lambd, X, y)
    grad_desc( [alpha, beta, q], lambd, X, y, end_diff)
    
my_logistic()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


i = 0
Coefficients: [1, -0.0005, 0.6]
Gradient; [-7.05748346e-05 -1.66244680e+00  2.61714828e-03]

i = 0
Cost change 2.769165803329088e-07
Cost0 is 1032.9999291457757
Coefficients: [ 1.00000000e+00 -4.99833755e-04  6.00000000e-01]
Gradient; [-7.08517321e-05 -1.66898393e+00  2.62666102e-03]

i = 1
Cost change 2.791009592328919e-07
Cost1 is 1032.9999288666747
Coefficients: [ 1.00000000e+00 -4.99666857e-04  5.99999999e-01]
Gradient; [-7.11308136e-05 -1.67557268e+00  2.63624578e-03]

i = 2
Cost change 2.813110313582001e-07
Cost2 is 1032.9999285853637
Coefficients: [ 1.00000000e+00 -4.99499300e-04  5.99999999e-01]
Gradient; [-7.14121051e-05 -1.68221367e+00  2.64590339e-03]

i = 3
Cost change 2.835477062035352e-07
Cost3 is 1032.999928301816
Coefficients: [ 1.00000000e+00 -4.99331078e-04  5.99999999e-01]
Gradient; [-7.16956329e-05 -1.68890752e+00  2.65563468e-03]

i = 4
Cost change 2.8581098376889713e-07
Cost4 is 1032.999928016005
Coefficients: [ 1.00000000e+00 -4.99162188e-04  5.99999999e-01

i = 45
Cost change 4.082769464730518e-07
Cost45 is 1032.9999139490913
Coefficients: [ 1.00000000e+00 -4.91586216e-04  5.99999987e-01]
Gradient; [-8.60472551e-05 -2.02781923e+00  3.14441701e-03]

i = 46
Cost change 4.1219232116418425e-07
Cost46 is 1032.999913536899
Coefficients: [ 1.00000000e+00 -4.91383435e-04  5.99999987e-01]
Gradient; [-8.64594129e-05 -2.03755454e+00  3.15835085e-03]

i = 47
Cost change 4.161645392741775e-07
Cost47 is 1032.9999131207344
Coefficients: [ 1.00000000e+00 -4.91179679e-04  5.99999986e-01]
Gradient; [-8.68755423e-05 -2.04738377e+00  3.17241345e-03]

i = 48
Cost change 4.2019405555038247e-07
Cost48 is 1032.9999127005403
Coefficients: [ 1.00000000e+00 -4.90974941e-04  5.99999986e-01]
Gradient; [-8.72957010e-05 -2.05730830e+00  3.18660664e-03]

i = 49
Cost change 4.242829163558781e-07
Cost49 is 1032.9999122762574
Coefficients: [ 1.00000000e+00 -4.90769210e-04  5.99999986e-01]
Gradient; [-8.77199477e-05 -2.06732952e+00  3.20093226e-03]

i = 50
Cost change 4.284

i = 88
Cost change 6.455229595303535e-07
Cost88 is 1032.9998917550693
Coefficients: [ 1.00000000e+00 -4.81851694e-04  5.99999972e-01]
Gradient; [-1.08239191e-04 -2.55215928e+00  3.88750546e-03]

i = 89
Cost change 6.533214218507055e-07
Cost89 is 1032.9998911017478
Coefficients: [ 1.00000000e+00 -4.81596478e-04  5.99999971e-01]
Gradient; [-1.08892444e-04 -2.56759862e+00  3.90917465e-03]

i = 90
Cost change 6.612617653445341e-07
Cost90 is 1032.999890440486
Coefficients: [ 1.00000000e+00 -4.81339719e-04  5.99999971e-01]
Gradient; [-1.09553636e-04 -2.58322587e+00  3.93109615e-03]

i = 91
Cost change 6.693483101116726e-07
Cost91 is 1032.9998897711378
Coefficients: [ 1.00000000e+00 -4.81081396e-04  5.99999971e-01]
Gradient; [-1.10222914e-04 -2.59904445e+00  3.95327446e-03]

i = 92
Cost change 6.77583557262551e-07
Cost92 is 1032.9998890935542
Coefficients: [ 1.00000000e+00 -4.80821492e-04  5.99999970e-01]
Gradient; [-1.10900425e-04 -2.61505791e+00  3.97571418e-03]

i = 93
Cost change 6.859720


i = 131
Cost change 1.1705251381499693e-06
Cost131 is 1032.9998541132886
Coefficients: [ 1.00000000e+00 -4.69198623e-04  5.99999953e-01]
Gradient; [-1.45876384e-04 -3.44206877e+00  5.11996042e-03]

i = 132
Cost change 1.189622025776771e-06
Cost132 is 1032.9998529236666
Coefficients: [ 1.00000000e+00 -4.68854416e-04  5.99999952e-01]
Gradient; [-1.47065840e-04 -3.47020410e+00  5.15842877e-03]

i = 133
Cost change 1.20918957691174e-06
Cost133 is 1032.999851714477
Coefficients: [ 1.00000000e+00 -4.68507395e-04  5.99999952e-01]
Gradient; [-1.48274859e-04 -3.49880284e+00  5.19750245e-03]

i = 134
Cost change 1.2292441624595085e-06
Cost134 is 1032.9998504852329
Coefficients: [ 1.00000000e+00 -4.68157515e-04  5.99999951e-01]
Gradient; [-1.49503928e-04 -3.52787653e+00  5.23719607e-03]

i = 135
Cost change 1.2498007890826557e-06
Cost135 is 1032.999849235432
Coefficients: [ 1.00000000e+00 -4.67804727e-04  5.99999951e-01]
Gradient; [-1.50753550e-04 -3.55743708e+00  5.27752469e-03]

i = 136
Cost c

Gradient; [-2.20925969e-04 -5.21840818e+00  7.50170659e-03]

i = 174
Cost change 2.7400774342822842e-06
Cost174 is 1032.999776310578
Coefficients: [ 1.00000000e+00 -4.51092182e-04  5.99999926e-01]
Gradient; [-2.23665472e-04 -5.28328771e+00  7.58712564e-03]

i = 175
Cost change 2.808851832014625e-06
Cost175 is 1032.9997735017262
Coefficients: [ 1.00000000e+00 -4.50563854e-04  5.99999926e-01]
Gradient; [-2.26473729e-04 -5.34979803e+00  7.67459135e-03]

i = 176
Cost change 2.8802448923670454e-06
Cost176 is 1032.9997706214813
Coefficients: [ 1.00000000e+00 -4.50028874e-04  5.99999925e-01]
Gradient; [-2.29353355e-04 -5.41800133e+00  7.76417914e-03]

i = 177
Cost change 2.954391675302759e-06
Cost177 is 1032.9997676670896
Coefficients: [ 1.00000000e+00 -4.49487074e-04  5.99999924e-01]
Gradient; [-2.32307105e-04 -5.48796298e+00  7.85596822e-03]

i = 178
Cost change 3.0314361083583208e-06
Cost178 is 1032.9997646356535
Coefficients: [ 1.00000000e+00 -4.48938277e-04  5.99999923e-01]
Gradient; [-2

Gradient; [-4.78650367e-04 -1.13306987e+01  1.52282070e-02]

i = 218
Cost change 1.3012688214075752e-05
Cost218 is 1032.999508229897
Coefficients: [ 1.0000000e+00 -4.1777395e-04  5.9999988e-01]
Gradient; [-4.91657255e-04 -1.16395560e+01  1.56053767e-02]

i = 219
Cost change 1.3736842674916261e-05
Cost219 is 1032.9994944930543
Coefficients: [ 1.00000000e+00 -4.16609994e-04  5.99999878e-01]
Gradient; [-5.05387812e-04 -1.19656308e+01  1.60024843e-02]

i = 220
Cost change 1.4522946230499656e-05
Cost220 is 1032.999479970108
Coefficients: [ 1.00000000e+00 -4.15413431e-04  5.99999877e-01]
Gradient; [-5.19903931e-04 -1.23103980e+01  1.64211740e-02]

i = 221
Cost change 1.537824982733582e-05
Cost221 is 1032.9994645918582
Coefficients: [ 1.00000000e+00 -4.14182391e-04  5.99999875e-01]
Gradient; [-5.35274747e-04 -1.26755052e+01  1.68632768e-02]

i = 222
Cost change 1.631109375921369e-05
Cost222 is 1032.9994482807645
Coefficients: [ 1.00000000e+00 -4.12914841e-04  5.99999874e-01]
Gradient; [-5.515

In [1]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import sklearn
from sklearn.model_selection import train_test_split
import random

def df2X(df, lag):
    '''
        Getting table and answer for training lag years ago
    '''
    #removing rows with an answer NaN
#     Y = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag]
#     #Y = df.iloc[:,4 + lag]#['2020/21']
#     X = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag + 1:]
#     #X = df.iloc[:,4 + lag + 1:]
    if lag > 0:
        # Checking if an answer is not NaN, and writing correct data to the table
        Y = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag]
        X = df[~np.isnan(df.iloc[:,4 + lag])].iloc[:,4 + lag + 1:]
        y = [1 if x < 100000 else 0 for x in Y]
    else:
        # No need to check if lag=0, this data are used for prediction so no y is available
        X = df.iloc[:,4 + lag + 1:]
        y = 'not used'
    for column_name in X.columns:
        X[column_name] *= 100/total_players[column_name]
    X.columns = range(len(X.columns))
    #To make an equal number of columns
    for i in reversed(range(max(lag - 1, 0))):
        X[13 - i] = [np.nan]*len(X)
    if lag == 0:
        del X[14]
    #X = (X-X.mean())/X.std()
    return X, y

def TablePrinter(df, row_num):
    styler = df.loc[:row_num].style
    styler.set_table_attributes('class="DataTable"')
    Rank_html = styler.render().replace("\n", "")

    with open("html/Ranking/Ranking.html", 'w', encoding="utf-8") as file:
        file.write(Rank_html)
        
seed = 1000

df1 = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)
df2 = pd.read_csv('Manager_ranking_500k-1010k.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)
df3 = pd.read_csv('Top_Manager_ranking.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)

df = df1.append(df2, ignore_index=True, sort=False)
df_all = df.append(df3, ignore_index=True, sort=False)
df = df_all.drop_duplicates(subset=['id']).reset_index(drop=True)

print(f'Rows number df: {len(df)}')
#df = df[df.count(axis=1)==20].reset_index()
#df = df.fillna(10000000)

total_players = {'2020/21': 8240322, '2019/20': 7628968, '2018/19': 6324237, '2017/18': 5910135, '2016/17': 4503345,
                 '2015/16': 3734001, '2014/15': 3502998, '2013/14': 3218998, '2012/13': 2608634, '2011/12': 2510000,
                 '2010/11': 2350000, '2009/10': 2100000, '2008/09': 1950000, '2007/08': 1700000, '2006/07': 1270000}

# Y = df['2020/21']
# y = [1 if x < 10000 else 0 for x in Y]
# X = df.loc[:,'2019/20':]

# #X = X.fillna(10000000)
# for column_name in X.columns:
#     X[column_name] *= 100/total_players[column_name] 
#     #X[column_name + '_NA'] = [ 1 if x == 10000000 else 0 for x in X[column_name]]
# print(f'Columns in X: {len(X.columns)}')
# X = (X-X.mean())/X.std()
X = pd.DataFrame()
y = np.array([])

# Sum up all data for different lags
for i in range(1,13):
    X_i, y_i = df2X(df, i)
    X = X.append(X_i, ignore_index=True, sort=False)
    y = np.array(list(y) + y_i)
    print(f'lag: {i}')
    #print(f'Columns in X: {len(X.columns)}')
#X, y = df2X(df, 1)






#makes equal number of '1' and '0'
X['answer'] = y
X_1 = X[X['answer']==1].reset_index(drop=True)
X_0 = X[X['answer']==0].reset_index(drop=True)

print(f'Number of successes {sum(y)}')
random.seed(1000)
#X_add = X_0.loc[random.sample(list(X_0.index), sum(y)*5), :].reset_index(drop=True)
#X = X_1.append(X_add, ignore_index=True, sort=False).reset_index(drop=True)
y = X['answer']
del X['answer']




print(f'Rows number X: {len(X)}')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
#X_train, X_test, y_train, y_test = X,X,y,y
print(f'Length Train: {len(X_train)}, Length Test: {len(X_test)}')

cat = CatBoostClassifier(random_state=seed, verbose=False, l2_leaf_reg=2)
cat.fit(X_train, y_train)   #Used for testing
#cat.fit(X, y) 

predict = cat.predict(X_train)
predict_proba = cat.predict_proba(X_train)
print(f'Train F1: {sklearn.metrics.fbeta_score(y_train, predict, 0.5)}')
print(f'Train roc_auc_score: {sklearn.metrics.roc_auc_score(y_train, pd.DataFrame(predict_proba)[1])}')

predict = cat.predict(X_test)
predict_proba = cat.predict_proba(X_test)
print(f'Test F1: {sklearn.metrics.fbeta_score(y_test, predict, 0.5)}')
print(f'Test roc_auc_score: {sklearn.metrics.roc_auc_score(y_test, pd.DataFrame(predict_proba)[1])}')

predict = cat.predict(df2X(df, 0)[0])
predict_proba = cat.predict_proba(df2X(df, 0)[0])
df_sorted = df.copy()
df_sorted['predict'] = predict
df_sorted['probability'] = pd.DataFrame(predict_proba)[1]
df_sorted = df_sorted.sort_values('probability',ascending=False).reset_index(drop=True)
df_sorted.loc[:10000].to_csv("rating.csv", index=False)# LIMIT AS OTHERWIZE MORE THAN 100MB!!!!!!!!!!!!!!!
TablePrinter(df_sorted, 999)
df_sorted.head(50)

Rows number df: 1052296
lag: 1
lag: 2
lag: 3
lag: 4
lag: 5
lag: 6
lag: 7
lag: 8
lag: 9
lag: 10
lag: 11
lag: 12
Number of successes 477745
Rows number X: 4927650
Length Train: 4434885, Length Test: 492765


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass beta=0.5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Train F1: 0.2677730376805886
Train roc_auc_score: 0.809664163058031


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass beta=0.5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Test F1: 0.2652988332407334
Test roc_auc_score: 0.8095303212095161


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,predict,probability
0,440328.0,Föbio Borges,Clichy's Cleansheets,Portugal,Southampton,1828.0,1965.0,95.0,671.0,1145.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.867805
1,331325.0,Sam Bristow,Gibbs-White MVP,England,Leeds,8215514.0,12517.0,458.0,4582.0,32250.0,...,51666.0,48088.0,8491.0,2007.0,6517.0,1818991.0,NaN,NaN,1,0.865101
2,33.0,Adnan Hajrulahovi?¯õ?,KDBinho,Slovenia,Liverpool,3240.0,9277.0,237.0,2799.0,3596.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.853258
3,611416.0,Mark Antoncich,Blixen FC,South Africa,Liverpool,3524.0,2840.0,2271.0,1938.0,136071.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.842686
4,1627870.0,Matt Corbidge,Yeezy Taught Me,Wales,NaN,5895.0,16655.0,4453.0,1463.0,1927.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.840328
5,10099.0,Cak Juris,Laskar Jember Utara¢?÷,Indonesia,Spurs,1632.0,7850.0,62157.0,5632.0,4421.0,...,8756.0,1092.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0.818162
6,3347.0,Morten Tveito,FC Gribben,Norway,Liverpool,838.0,8688.0,421.0,1051.0,20952.0,...,13456.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.816653
7,1601.0,Stig Hermansen,Tufs,Norway,West Brom,12428.0,7224.0,2168.0,3186.0,85213.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.815555
8,184407.0,Anders Lium,AFC Helvete,Norway,NaN,7345.0,2771.0,66408.0,52.0,13305.0,...,227775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.815438
9,162758.0,Ilmari Juva,Rahapallo,Finland,NaN,592.0,23521.0,5279.0,4960.0,6875.0,...,214944.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.813934


In [27]:
# df2X(df, 6)
# df_sorted[df_sorted['name'].str.contains("Suther")==True]
def TablePrinter(df, row_num):
    styler = df.loc[:row_num].style
    styler.set_table_attributes('class="DataTable"')
    Rank_html = styler.render().replace("\n", "")

    with open("html/Ranking/Ranking.html", 'w', encoding="utf-8") as file:
        file.write(Rank_html)
# Rank_html
TablePrinter(df_sorted, 999)

In [70]:
print ('Так плохо:', 'p/\nic')
print ('А с экранированием хорошо:', 'p/\\nic')

with open('Manager_ranking_0-500k.csv', 'r', encoding= 'unicode_escape') as f:
    text = f.read().replace('\\n','')
with open('Manager_ranking_0-500k.csv', 'w', encoding= 'unicode_escape') as f:
    f.write(text)

Так плохо: p/
ic
А с экранированием хорошо: p/\nic


In [76]:
df_sorted[df_sorted['2020/21'] == 1828]
#len(df)
#df_all['id'] = pd.to_numeric(df_all['id'], downcast="integer")
df_all.drop_duplicates(subset=['id'])[df_all['2020/21'] == 1828]
# 'USA' in df_all['id']

# P/\nic MЇх?nster
#df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,2015/16,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07
440327,440328.0,Föbio Borges,Clichy's Cleansheets,Portugal,Southampton,1828.0,1965.0,95.0,671.0,1145.0,4682.0,17996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579991,579992.0,Jan Are Bjerken?s,Juve,Norway,Newcastle,1828.0,113349.0,40031.0,183688.0,35698.0,22869.0,51425.0,60705.0,52423.0,23745.0,16844.0,5433.0,NaN,NaN,NaN
1055721,3781024.0,Gill Van Puyvelde,AndrÃ©s Mendoza,Belgium,Man Utd,1828.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_all['id'][484752]
df_all[484751:]

,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,2015/16,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07
484751,484752.0,Prasanth Nair,P/!!nic M¯õ?nster,USA,NaN,1068361.0,618649.0,1535881.0,815933.0,NaN,NaN,NaN,NaN,NaN,1593077.0,210620.0,2043930.0,NaN,NaN,NaN
484752,484753.0,Ahmed Hemdan,Manchester city,Egypt,NaN,4563328.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484753,484754.0,Bassem Bassiouny,Chilly Twoface,Egypt,NaN,7644132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484754,484755.0,Ahmed ELgarm,Elgarm,Egypt,NaN,5399810.0,7520447.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484755,484756.0,kyle donaghy,KevinWebster÷¢??¢?÷sJacket,England,NaN,2964863.0,4579036.0,1071159.0,NaN,NaN,NaN,NaN,NaN,NaN,818734.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109995,3501031.0,Andreas GÃ¼nther,GÃ¼nther FC,Norway,NaN,8113.0,2056954.0,854622.0,157310.0,237469.0,NaN,NaN,2118189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1109996,2151863.0,Joanne Mccarthy,SMA Rovers,Ireland,Liverpool,18479.0,1214471.0,516175.0,128647.0,875774.0,1115601.0,109859.0,620283.0,1246527.0,NaN,NaN,NaN,NaN,NaN,NaN
1109997,359716.0,Ahmed Hesham,Ahmed Hesham,Egypt,Man City,214948.0,42100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1109998,3883889.0,Andrew Kerr,Balls up,England,Man Utd,261904.0,37500.0,569971.0,1059796.0,48925.0,2662307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
np.isnan(np.nan)
np.isnan(df.iloc[0,7])
np.nan < 1
df[~np.isnan(df['2016/17'])]
df[~np.isnan(df.iloc[:,4 + 1])].iloc[:,4 + 1]
print(len(X_0))
sum(y)

4548955


46671

In [14]:
df_sorted.head(20)
# df_sorted[df_sorted['name']=='Fѓцbio Borges']
# #df2X(df, 0)[0]
# df_sorted
#df2X(df, 0)[0]
#df_sorted.sum()
df_sorted[df_sorted['name']=='Tom Stephenson']

,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,predict,probability
639819,129818,Tom Stephenson,Badgers9,England,Wolves,1225.0,375.0,1841.0,29590.0,4886.0,...,22796.0,4701.0,18733.0,12105.0,35834.0,11669.0,261173.0,NaN,0,0.268400
573822,63821,Tom Stephenson,The Yorkshire Pirlo,England,Leeds,237764.0,1553812.0,146801.0,409796.0,705534.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.023668
349314,849314,Tom Stephenson,Leeds Leeds Leeds,England,Spurs,1763953.0,179237.0,111181.0,47728.0,905928.0,...,726379.0,24065.0,23152.0,11065.0,93266.0,147.0,73818.0,486.0,0,0.013466
308300,808300,Tom Stephenson,T. Boone Pickens,England,Man City,514710.0,2977553.0,542506.0,1237583.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.008947
324220,824220,Tom Stephenson,Winnersh Rangers,England,NaN,1185046.0,1182945.0,3312729.0,NaN,NaN,...,1990519.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.005924
955242,445241,Tom Stephenson,Mephistopheles,England,Chelsea,2962874.0,1234321.0,907603.0,788318.0,442622.0,...,458907.0,462073.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.001674


In [6]:
df2X(df, 0)[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.255384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.612146,-0.426108,2.000135,1.157442,-0.032467,1.111041,0.026725,-0.795226,0.247625,1.052653,1.212808,-0.136814,2.714687,NaN
2,1.393648,2.358557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.817484,1.601736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.928990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009998,-0.426688,-1.008382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009999,1.430919,1.725720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010000,1.140500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010001,0.582164,0.213264,1.519281,0.521684,2.306198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_sorted.head(20)
df.mean()

2020/21        2.425109e+06
2019/20        2.220926e+06
2018/19        1.994808e+06
2017/18        1.817956e+06
2016/17        1.369274e+06
2015/16        1.114764e+06
2014/15        1.032212e+06
2013/14        9.688040e+05
2012/13        8.003145e+05
2011/12        7.972330e+05
2010/11        6.570688e+05
2009/10        5.846533e+05
2008/09        4.805558e+05
2007/08        4.232473e+05
2006/07        2.908257e+05
predict        4.455432e-05
probability    5.138853e-03
dtype: float64

In [7]:
total_players = {'2020/21': 8240322, '2019/20': 7628968, '2018/19':6324237, '2017/18': 5910135, '2016/17': 4503345,
                 '2015/16': 3734001, '2014/15': 3502998, '2013/14': 3218998, '2012/13': 2608634, 
                 '2011/12':2510000, '2010/11': 2350000, '2009/10':2100000,
                 '2008/09': 1950000, '2007/08':1700000, '2006/07': 1270000}
total_players

{'2020/21': 8240322,
 '2019/20': 7628968,
 '2018/19': 6324237,
 '2017/18': 5910135,
 '2016/17': 4503345,
 '2015/16': 3734001,
 '2014/15': 3502998,
 '2013/14': 3218998,
 '2012/13': 2608634,
 '2011/12': 2510000,
 '2010/11': 2350000,
 '2009/10': 2100000,
 '2008/09': 1950000,
 '2007/08': 1700000,
 '2006/07': 1270000}